In [ ]:
# @title Install and Setup
!pip install onnx  opencv-python insightface gdown onnxruntime rich
#!pip install onnxruntime-gpu --extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/
import datetime
import numpy as np
import os, subprocess, re, glob, cv2
import os.path as osp
import insightface
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image
import matplotlib.pyplot as plt
import gdown
from PIL import Image
from google.colab import files
from IPython.display import clear_output
from rich.progress import track

def upload_files():
    uploaded = files.upload()
    return list(uploaded.keys())

#/root/.insightface/models/buffalo_l.zip
app = FaceAnalysis(name='buffalo_l',providers=['CUDAExecutionProvider']) # , root=str('models/insightface')
app.prepare(ctx_id=0, det_size=(640, 640))
# Import the library
import gdown

# Google Drive file ID
file_id = '1krOLgjW2tAPaqV-Bw4YALz0xT5zlb5HF'
# Construct the download URL
url = f'https://drive.google.com/uc?id={file_id}'
# Download the file
gdown.download(url, 'inswapper_128.onnx', quiet=False)

swapper = insightface.model_zoo.get_model('inswapper_128.onnx',download=False,download_zip=False)

os.system("mkdir output")
os.system("mkdir frame")

def ffmpegp(command,desc=''):
  process = subprocess.Popen(command.split(), stderr=subprocess.PIPE, universal_newlines=True)
  for line in track(iter(process.stderr.readline,''),description=desc):
    if "time=" in line:
      time = re.findall(r"time=\s*(\d{2}:\d{2}:\d{2}\.\d{2})", line)

def facecompare(face1,face2):
  feat1 = face1.normed_embedding
  feat2 = face2.normed_embedding
  similarity = np.dot(feat1, feat2)
  threshold = 0.5
  if similarity > threshold:
      return True
  else:
      return False

def faceswapdir(face_image,path=".",face_id=0,fcfrmid=0):
  img1 = cv2.imread(face_image)
  lst = os.listdir(path)
  lst.sort()
  img_lst = [cv2.imread(f"{path}/{img}") for img in lst]
  base_face = app.get(img1)[0]
  face_lst =[app.get(face)[face_id] if app.get(face) else None  for face in img_lst]
  out_img = [swapper.get(swap.copy(), face_lst[i], base_face, paste_back=True)  if face_lst[i] is not None  and  facecompare(face_lst[i],face_lst[fcfrmid]) else img_lst[i] for i,swap in enumerate(track(img_lst, description="Swap frames "))]
  swapped_img1_rgb = [cv2.cvtColor(swapped, cv2.COLOR_BGR2RGB) for swapped in out_img]

  for i,swapped2 in enumerate(swapped_img1_rgb):
    Image.fromarray(np.uint8(swapped2)).save(f"output/{i+1:06}.jpg")

def faceswapdirold(face_image,path=".",face_id=0):
  img1 = cv2.imread(face_image)
  lst = os.listdir(path)
  lst.sort()
  img_lst = [cv2.imread(f"{path}/{img}") for img in lst]
  base_face = app.get(img1)[0]
  face_lst =[app.get(face)[0] if app.get(face) else None  for face in img_lst]
  out_img = [swapper.get(swap.copy(), face_lst[i], base_face, paste_back=True)  if face_lst[i] is not None  else img_lst[i] for i,swap in enumerate(track(img_lst, description="Swap frames "))]
  swapped_img1_rgb = [cv2.cvtColor(swapped, cv2.COLOR_BGR2RGB) for swapped in out_img]

  for i,swapped2 in enumerate(swapped_img1_rgb):
    Image.fromarray(np.uint8(swapped2)).save(f"output/{i+1:06}.jpg")

clear_output()

In [ ]:
# @title Run FaceSwap Image
!rm -r /content/allpic
face_image = upload_files()
!mkdir /content/allpic
%cd /content/allpic
image_files = upload_files()
%cd /content

faceswapdir(face_image[0],"/content/allpic")

In [ ]:
# @title Only face upload
face_image = upload_files()
!cp "{face_image[0]}" 7.jpg
print('insightface', insightface.__version__)
print('numpy', np.__version__)

# Check that InsightFace version is at least 0.7
assert float('.'.join(insightface.__version__.split('.')[:2])) >= float('0.7')
clear_output()

In [ ]:
# @title Only Video upload
video_file = upload_files()
!mv "{video_file[0]}" iv.mp4
clear_output()

In [ ]:
# @title Run FaceSwap Video
fcid = 75 # @param {"type":"integer"}
ffmpegp('ffmpeg -hide_banner -loglevel error  -i iv.mp4 -q:a 0 -map a ia.mp3','Extract Audio ')
print('Completed.')
ffmpegp('ffmpeg -hide_banner -loglevel error  -hwaccel auto -i iv.mp4 -q:v 18 -pix_fmt rgb24 -vf fps=25  /content/frame/%06d.jpg','Extract frames ')
print('Completed.')
faceswapdir(face_image[0],"/content/frame",fcfrmid=fcid)
ffmpegp('ffmpeg -hide_banner -loglevel error  -hwaccel auto -r 25 -i /content/output/%06d.jpg -vcodec libx264 -crf 18  -pix_fmt yuv420p   ov.mp4','Fraems to Video ')
print('Completed.')
ffmpegp('ffmpeg -hide_banner -loglevel error  -i ov.mp4 -i ia.mp3 -c:v copy -c:a libmp3lame  video.mp4','Merge Video and Audio ')
print('Completed.')

In [ ]:
# @title Extract A-B
ffmpeg('ffmpeg -ss 00:01:00 -to 00:02:00 -i input.mp4 -c copy output.mp4','Cut A-B ')
#!ffmpeg -hide_banner -loglevel error  -i iv.mp4 -q:a 0 -map a ia.mp3
#!ffmpeg -hide_banner -loglevel error  -hwaccel auto -i iv.mp4 -q:v 18 -pix_fmt rgb24 -vf fps=25  /content/frame/%06d.jpg
clear_output()

In [ ]:
# @title Remove all files  from frame/output folder
!rm -r /content/frame
!rm -r /content/output
!mkdir /content/output
!mkdir /content/frame

In [41]:
import subprocess
import re
from rich.progress import Progress

# Assume you have the total_duration in seconds
total_duration = 120.0  # Example: 2 minutes

with Progress() as progress:
    task_id = progress.add_task("[green]Processing...", total=total_duration)

    command = "ffmpeg -i iv.mp4 ov.mp4"
    process = subprocess.Popen(command.split(), stderr=subprocess.PIPE, universal_newlines=True)

    for line in iter(process.stderr.readline,''):
      if 'time' in line
      h, m, s, ms = re.findall('(\d{2}):(\d{2}):(\d{2}).(\d{2})', line)[0]
      if h and m and s :
        cts = int(h)*3600 +  int(m)*60 + int(s)
      progress.update(task_id, completed=cts)

    process.wait()

Output()

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers

IndexError: list index out of range